Related to user story: [SP11-Item04: General Data Wrapper PoC](https://gitlab.inria.fr/fedbiomed/fedbiomed/-/issues/164)

## Tabular dataset

Workflow of data pre processing:

1. Columns name should be shared with the researcher
2. Data format file to be filled by clinicians.
3. Specify if missing data are allowed for a given columns (Exception). The file will be used for data verification during FL pre-processing,
4. Outlier verification for quantitative data, continuous and discrete, and for dates (Critical warning),
5. Missing data imputation by local mean (or optional NN), or majority voting for discrete labels. Give warnings when missing data are found (for verification a posteriori).
6. Give critical warning when too many missing are found (>50%),
7. Verify that number of available data is greater then minimum required (Error)

Critical warnings have different levels of disclosure to the researcher (1) only the warning, 2) type of warning, 3) type of warning and column affected).

In [60]:
!pip install prettytable

In [1]:
#1. load  a single view dataset


import pandas as pd
import pprint
import csv
import numpy as np
from typing import List, Tuple, Union, Dict, Any, Iterator, Optional, Callable
import os
from tabulate import tabulate
import json



In [2]:
from enum import Enum, auto

class ExcelSignatures(Enum):
    XLSX = (b'\x50\x4B\x05\x06', 2, -22, 4)
    LSX1 = (b'\x09\x08\x10\x00\x00\x06\x05\x00', 0, 512, 8)
    LSX2 = (b'\x09\x08\x10\x00\x00\x06\x05\x00', 0, 1536, 8)
    LSX3 = (b'\x09\x08\x10\x00\x00\x06\x05\x00', 0, 2048, 8)
    
    def __init__(self, sig, whence, offset, size):
        self._sig = sig
        self._whence = whence
        self._offset = offset
        self._size = size

    @property 
    def signature(self) -> bytes:
        return self._sig
    
    @property
    def whence(self) -> int:
        return self._whence
    
    @property
    def offset(self) -> int:
        return self._offset
    
    @property
    def size(self) -> int:
        return self._size



def load_tabular_datasets(path:str) -> Dict[str, pd.DataFrame]:
    tabular_datasets = {}

    if os.path.isdir(path):
        print('directory found')
        _is_folder = True
        
        _tabular_data_files = os.listdir(path)
    else:
        print('file found')
        _is_folder = False
        _tabular_data_files = (path,)
        
    for tabular_data_file in _tabular_data_files:
        if _is_folder:
            tabular_data_file = os.path.join(path, tabular_data_file)
        
        _is_excel = excel_sniffer(tabular_data_file)
        _csv_delimiter, _csv_header = csv_sniffer(tabular_data_file)
        _view_name = os.path.basename(tabular_data_file)
        if _is_excel:
            tabular_datasets[_view_name] = load_excel_file(tabular_data_file)
        elif _csv_delimiter is not None:
            tabular_datasets[_view_name] = load_csv_file(tabular_data_file,
                                                               _csv_delimiter, 
                                                               _csv_header)
        else:
            print(f'warning: cannot parse {tabular_data_file}: not a tabular data file')
        
    return tabular_datasets

def load_csv_file(path:str, delimiter:str, header:int) -> pd.DataFrame:
    try:
        dataframe = pd.read_csv(path, delimiter=delimiter, header=header)
    except csv.Error as err:
        print('err', err, 'in file', path)
            
    return dataframe

#https://stackoverflow.com/questions/23515791/how-to-check-the-uploaded-file-is-csv-or-xls-in-python/23515973




def load_excel_file(path:str, sheet_name: Union[str, int]=0) -> pd.DataFrame:
    """May rely on openpyxl package"""
    #with open(path, 'r') as excl:
    #    _c = csv.DictReader(excl, dialect=csv.excel_tab)
    #    _delimiter = _c.dialect.delimiter
    
    dataframe = pd.read_excel(path, sheet_name=sheet_name)
    return dataframe


def excel_sniffer(path: str) -> bool:
    
    for excel_sig in ExcelSignatures:
        with open(path, 'rb') as f:
            f.seek(excel_sig.offset, excel_sig.whence)
            bytes = f.read(excel_sig.size)

            if bytes == excel_sig.signature:
                return True
            else:
                return False
            

def csv_sniffer(path:str) :
        
    with open(path, 'r') as csvfile:
        try:
            # do some operation on file using sniffer to make sure considered file
            # is a CSV file
            dialect = csv.Sniffer().sniff(csvfile.readline())
            delimiter = dialect.delimiter
            dialect.lineterminator
            has_header = csv.Sniffer().has_header(csvfile.readline())
            if has_header:
                header = 0
            else:
                header = None
        except (csv.Error, UnicodeDecodeError) as err:
            delimiter, header = None, None
            print('err', err, 'in file', path)
    return delimiter, header

In [52]:
!pip install openpyxl

In [30]:
load_tabular_datasets('../../Exceltest.xlsx')

file found
err 'utf-8' codec can't decode byte 0x8c in position 15: invalid start byte in file ../../Exceltest.xlsx


{'Exceltest.xlsx':    ID   Age Eligibility
 0    1   45           Y
 1    2   45           Y
 2    3   33           N
 3    4   54           Y
 4    5   45           Y
 5    6   54         NaN
 6    7   34           N
 7    8   54         NaN
 8    9   45         NaN
 9   10   44           Y}

In [16]:
single_view_dataset = load_tabular_datasets(r'/user/ybouilla/home/Documents/data/pseudo_adni_mod/pseudo_adni_mod.csv')

file found


In [33]:
multi_view_dataframe =  load_tabular_datasets('test7')
multi_view_dataframe

directory found


{'file1':      a   e   i   o      0      1      2      3                 time  pressure  \
 0   48  98  65   5  False   True  False  False  2018-01-01 00:00:00  0.088082   
 1   87  83  13  70   True  False   True  False  2018-01-01 01:00:00  0.774788   
 2   46  73  81  96  False  False  False   True  2018-01-01 02:00:00  0.514092   
 3   84  45  81  39  False   True   True   True  2018-01-01 03:00:00  0.832881   
 4   94  84   0  15  False   True  False  False  2018-01-01 04:00:00  0.696152   
 ..  ..  ..  ..  ..    ...    ...    ...    ...                  ...       ...   
 95  14  66  25  64   True   True   True  False  2018-01-04 23:00:00  0.295578   
 96  91  81  48  53  False  False   True   True  2018-01-05 00:00:00  0.474322   
 97  15  82  12  51   True   True   True   True  2018-01-05 01:00:00  0.927511   
 98  51  18   4  52  False  False   True   True  2018-01-05 02:00:00  0.494798   
 99  51  70  63  77  False  False   True  False  2018-01-05 03:00:00  0.316395   
 
     

Data format file to be filled by clinicians (step 2 int he workflow):

Data format file will be a dictionary specifying the type: 
* for single view datasets:
```{<feature_name>: {'data_type': <data_type>, 'type':<values_taken>, 'range': <value_range>}```
 * for multiview datatset
```{{<view_name>: <feature_name>: {'data_type': <data_type>, 'type':<values_taken>, 'range': <value_range>}}```

where
* `<view_name>` is the name of the view
* `<feature_name>` is the name of the feature
* `<data_type>` can be categorical or continuous or missing_data or datetime
* `<value_taken>` is the type of the value (eg int, char, float, signed, unsigned ...)
* `<value_range>` represent either a list of bounds, an upper or a lower bound, or None

In [17]:
# 3. create data format file

import numpy as np
from enum import Enum
import enum
import datetime

# the use of Enum classes will prevent incorrect combination of values
class QuantitativeDataType(Enum):
    CONTINUOUS = [float, np.float64]
    DISCRETE = [int, np.int64]

class CategoricalDataType(Enum):
    BOOLEAN = [bool]
    NUMERICAL = [float, int, np.float64, np.int64]
    CHARACTER = [str, object]
    
class KeyDataType(Enum):
    NUMERICAL = [int, np.int64]
    CHARACTER = [str, object]
    DATETIME = [pd.Timestamp,
                pd.Timedelta,
                pd.Period,
                datetime.datetime,
                np.datetime64]


class CustomDataType(Enum):
    """for demo purpose: here a custom datatype"""
    DISCRETE = [int, np.int64]
    CHARACTER = [str, object]
    
    
class DataType(Enum):
    """

    """
    # what about 
    KEY = KeyDataType
    QUANTITATIVE = QuantitativeDataType
    CATEGORICAL = CategoricalDataType
    DATETIME = [pd.Timestamp,
                pd.Timedelta,
                pd.Period,
                datetime.datetime,
                np.datetime64]
    CUSTOM = CustomDataType  # custom data type (should be defined by user)
    UNKNOWN = 'UNKNOWN'
    
    @staticmethod
    def get_names():
        return tuple(n for n, _ in DataType.__members__.items())

class MissingValueAllowedDefault(Enum):
    KEY = False
    QUANTITATIVE = True
    CATEGORICAL = True
    DATETIME = False
    
    @staticmethod
    def get_names():
        return tuple(n for n, _ in MissingValueAllowedDefault.__members__.items())
    
    
class DataTypeProperties(Enum):
    """Data Type possible modification (whithin CLI editing)"""
    CATEGORICAL = (False, False, True, False, True)
    QUANTITATIVE = (True, True, False, False, True)
    DATETIME = (True, True, False, True, False)
    UNKNOWN = (False, False, False, False, True)
    CUSTOM = (True, True, True, False, True)
    KEY = (True, True, False, True, False)

    def __init__(self,
                 lower_bound: bool,
                 upper_bound: bool,
                 set_of_values: bool,
                 date_format:bool,
                 allow_missing_values: bool):
        self._lower_bound = lower_bound
        self._upper_bound = upper_bound
        self._set_of_values = set_of_values
        self._date_format = date_format
        self._allow_missing_values = allow_missing_values
    
    @property
    def lower_bound(self):
        return self._lower_bound
    
    @property
    def upper_bound(self):
        return self._upper_bound
    
    @property
    def set_of_values(self):
        return self._set_of_values
    
    @property
    def date_format(self):
        return self._date_format
    
    @property
    def allow_missing_values(self):
        return self._allow_missing_values



In [20]:
ImputationMethods.MODE_IMPUTATION

<function __main__.impute_missing_values_mode(data: Union[pandas.core.series.Series, pandas.core.frame.DataFrame])>

In [29]:
isinstance(next(iter(DataType.DATETIME.value)), Enum)

False

In [4]:
def get_data_type(
                  #avail_data_types: enum.EnumMeta,
                  d_format: Enum,
                  d_type: type) ->  Tuple[Enum, List[Union[type, str]]]:
    # varibales initialisation
    present_d_types = []
    sub_d_type_format = d_format
    
    
    for avail_data_type in DataType:
        if d_format is avail_data_type:
            sub_dtypes = avail_data_type.value
            #if not isinstance(sub_dtypes, str) and hasattr(sub_dtypes, '__getitem__') and isinstance(sub_dtypes[0], Enum):
            if not isinstance(sub_dtypes, str):
                # check if dtype has subtypes
                #(eg if datatype is QUANTITATIVE, subtype will be CONTINOUS or DISCRETE)
                if isinstance(next(iter(sub_dtypes)), Enum):
                    
                    for sub_dtype in sub_dtypes:
                        if any(d_type == t for t in tuple(sub_dtype.value)):
                            present_d_types.append(d_type)
                            sub_d_type_format = sub_dtype
                            print(sub_dtype, d_type)
                else:
                    # case where datatype doesnot have subtypes, eg DATETIME
                    if any(d_type == t for t in sub_dtypes):
                        present_d_types.append(d_type)
                    sub_d_type_format = d_format
            else:
                # case where d_format is a string of character
                sub_d_type_format = d_format
    print(sub_d_type_format, '|', present_d_types)
    return  sub_d_type_format, present_d_types


def find_data_type(data_format_name: str, data_type_name: str=None) -> List[Enum]:
    """Retrieves from a given data_format and data_type,
    the corresponding Enum class describing data"""
    
    ## varible initialisation
    data_type = None
    _is_data_format_unrecognized = True
    _is_data_type_unrecognized = True
    
    _available_data_types = [t for t in DataType]
    
    for a_data_type in _available_data_types:
        if data_format_name == a_data_type.name:
            _is_data_format_unrecognized = False
            data_type = a_data_type
            
            for sub_type in a_data_type.value:
                
                if data_type_name is not None and  isinstance(sub_type, Enum):
                    # check if sub data type exist (it shouldnot if variable is UNKNOWN)
                    if data_type_name == sub_type.name:
                        
                        _is_data_type_unrecognized = False
                        data_type = sub_type
                
                    
                else:
                    _is_data_type_unrecognized = False
    # check for data formt file consistancy error
    if any((_is_data_format_unrecognized, _is_data_type_unrecognized)):
        if _is_data_format_unrecognized:
            raise ValueError(f'error: {data_format_name} not recognized as a valid data type')
        else:
            raise ValueError(f'error {data_type_name} not recognized as a valid data type')
            
    return data_type

In [5]:

def check_missing_data(column: pd.Series)->bool:
    is_missing_data = column.isna().any()
    return is_missing_data
df = pd.DataFrame({'w': [1, 2, 3, 4,  'jj', None]})
print(check_missing_data(df['w']))




True


CLI details:

1. open  csv file
2. for each columns in file ask type of variable or if variable should be excluded
3. automatically detect the type given values in columns 
4. ask for each columns if missing data are allowed


eg :

assume a column is of type discrete with integers


1. user select it is quantitative
2. then system will label it as quantitative-discrete


**Question** : do we want an auto selection parameter choice?

In [6]:
def get_yes_no_msg() -> str:
    msg_yes_or_no_question = '1) YES\n2) NO\n'   
    return msg_yes_or_no_question

def parse_yes_no_msg(resp: str) -> bool:
    """implements logic to parse yes or no msg"""
    yes_or_no_question_key = {'1': True,
                    '2': False}
    return yes_or_no_question_key.get(resp)

def get_data_type_selection_msg(available_data_type:List[Enum],
                               ign_msg: str = 'ignore this column') ->Tuple[str, int]:
    
    
    n_available_data_type = len(available_data_type)
    msg = ''

    
    for i, dtype in enumerate(available_data_type):
        msg += '%d) %s \n' %  (i+1, dtype.name)
    
    ignoring_key = i+2  # add ingoring entry
    msg += f'%d) %s\n' % (ignoring_key, ign_msg)
    
    
    
    return msg, ignoring_key

def unique(iterable: Iterator, number: bool = False) -> int:
    """returns number of unique values"""
    set_of_values = set(iterable)
    if number:
        return len(set_of_values)
    else:
        return set_of_values

In [48]:
from functools import partial

# imputation methods

def impute_missing_values_mean(data):
    try:
        if type(data) == pd.core.frame.DataFrame:
            for col in data.columns:
                if (data[col].isnull().sum()>0):
                    if any(data[col].dtype in x2 for x2 in  [x.value for x in QuantitativeDataType]):
                        data[col].fillna(value=data[col].mean(),inplace=True)
        else:
            data = data.fillna(data.mean())
        return data
    except Exception as err:
        print(err)
        print('Error encountered in loading data file')
    
 #Categorical Data
def impute_missing_values_mode(data: Union[pd.Series, pd.DataFrame]):
    try:
        if type(data) == pd.core.frame.DataFrame:
            for col in data.columns:
                if (data[col].isnull().sum()>0):
                    categorical_data_type = [x.value for x in CategoricalDataType]
                    data_type_condition = any(data[col].dtype in x2 for x2 in categorical_data_type)
                    if data_type_condition:
                        print(col)
                        data[col].fillna(value=data[col].mode()[0],inplace=True)
                        
        else:
            data = data.fillna(data.value_counts().index[0])
            
        return data
    except Exception as err:
        print(err)
        print('Error encountered in imputing missing values - mode')
        
    # Impute missing values with KNN            

def impute_missing_values_knn(data,k=2):
    try:
        if type(data) == pd.core.frame.DataFrame:
            missing_cols = data.columns[data.isnull().any()]
            if len(missing_cols)>0:
                    imputer =KNNImputer(n_neighbors=k)
                    data = pd.DataFrame(imputer.fit_transform(data),columns=data.columns) 
        else:
            imputer =KNNImputer(n_neighbors=k)
            data =pd.DataFrame( (imputer.fit_transform(np.array(data).reshape(1,-1))).reshape(-1,1),columns=[data.name])                    

        return data  
    except Exception as err:
        print(err)
        print('Error encountered in imputing missing values - knn')

#Impute missing values with Interpolate
def impute_missing_values_interpolate(data):
    try:
        data_filled = data.interpolate()
            
        return data_filled
    except Exception as err:
        print(err)
        print('Error encountered in imputing missing values - interpolate')
        

class ImputationMethods(Enum):
    MEAN_IMPUTATION = partial(impute_missing_values_mean)
    MODE_IMPUTATION = partial(impute_missing_values_mode)
    KNN_IMPUTATION = partial(impute_missing_values_knn)
    INTERPOLATION_IMPUTATION = partial(impute_missing_values_interpolate)
    
    def __call__(self, *args):
        """method avoiding to specify `value` when using an enum class"""
        self.value(*args)


In [53]:
a, b = get_data_imputation_methods_msg()
len(b)

4

In [69]:
# CLI for clinicians for setting up data format file



In [56]:
def get_data_imputation_methods_msg() -> Tuple[str, Dict[str, str]]:
    msg = 'Please select the following method for filling missing values (if some are found)\n'
    
    #available_methods = [method for ethod in ImputationMethods]
    select_action = {}
    for i, method in enumerate(ImputationMethods):
        
        msg += '%d) %s\n' % (i+1, method.name)
        select_action[str(i)] = method.name

    # ignore key
    msg += '%d) No method\n' % (i + 2)
    select_action[str(i+1)] = None
    return msg, select_action
    
def no_methods(*kwargs):
    return None

def set_imputation_method(user_action: str, available_methods: Dict[str, Any]):
    pass
    
    
def get_from_user_dataframe_format_file(dataset: pd.DataFrame) -> Dict[str, Any]:
    ##
    # variable initialisation
    data_format_file = {}
    
    dataset_columns = dataset.columns
    dataset_columns_length = len(dataset_columns)
    
    
    available_data_type = [d_type for d_type in DataType]  # get all available data types
    
    for n_feature, feature in enumerate(dataset_columns):
        print(f'displaying first 10 values of feature {feature} (n_feature: {n_feature+1}/{dataset_columns_length})')
        #print(tabulate(dataset[feature].head(10).values()))
        pprint.pprint(dataset[feature].head(10))  # print first 10 lines of feature value
        print(f'number of differents samples: {unique(dataset[feature], number=True)} / total of samples: {dataset[feature].shape[0]}')
        
        msg_data_type_selection, ignoring_id = get_data_type_selection_msg(available_data_type)
        msg_data_type_selection = f'specify data type for {feature}:\n' + msg_data_type_selection
        
        # ask user about data type
        data_format_id = get_user_input(msg_data_type_selection,
                                       
                                       n_answers=ignoring_id)
        
        if int(data_format_id) > ignoring_id - 1:
            # case where user decide to ingore column: go to next iteration (next feature)
            print(f"Ignoring feature {feature}")
            continue
        else:
            # case where user selected a data type: add data type and info to the format file
            data_format = available_data_type[int(data_format_id)-1]
            data_type = dataset[feature].dtype
            n_data_type, types = get_data_type(data_format, data_type)
            
        # KEY and DATETIME type 
        if data_format is DataType.KEY or data_format is DataType.DATETIME:  
            # for these data type, missing values are disabled by default
            is_missing_values_allowed = False
        else: 
            # ask user if missing values are allowed for this specific variable
            ## message definition
            msg_yes_or_no_question = get_yes_no_msg()
            msg_data_imputation_methods, data_imputation_methods = get_data_imputation_methods_msg()
            n_data_imputation_method = len(data_imputation_methods)
            msg_yes_or_no_question = f'Allow {feature} to have missing values:\n' + msg_yes_or_no_question
            
            missing_values_user_selection = get_user_input(msg_yes_or_no_question,
                                                        n_answers=2)
            is_missing_values_allowed = parse_yes_no_msg(missing_values_user_selection)
            
            if is_missing_values_allowed:
                # let user select amputation method if missing data are allowed
                amputation_meth_user_selection = get_user_input(msg_data_imputation_methods,
                                                                n_answers=n_data_imputation_method)
                
                amputation_meth_selected = data_imputation_methods.get(amputation_meth_user_selection)
            else:
                # no amputation is required (because missing values are not required)
                amputation_meth_selected = None
                
        data_format_file[feature] = {'data_format': data_format.name,
                                     'data_type': n_data_type.name,
                                     'values': str(data_type),
                                     'is_missing_values': is_missing_values_allowed,
                                     'data_amputation_method': amputation_meth_selected
                                    }
            
    return data_format_file
            
def get_user_input(msg:str,  n_answers:int) -> str:
    """"""
    is_column_parsed = False
    while not is_column_parsed:
        #data_format_id = input(f'specify data type for {feature}:\n' + msg )
        resp = input(msg)
        if resp.isdigit() and int(resp) <= n_answers and int(resp)>0:
            # check if value passed by user is correct (if it is integer,
            # and whithin range [1, n_available_data_type])
            is_column_parsed = True

        else:
            print(f'error ! {resp} value not understood')
            
    return resp

In [50]:
### CLI to use when dataset is available


def get_from_user_multi_view_dataset_fromat_file(datasets: Dict[str, pd.DataFrame])-> Dict[str, pd.DataFrame]:
    
    data_format_files = {}
    
    for tabular_data_file in datasets.keys():
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        print(f"+++++++ Now parsing view: {tabular_data_file} +++++++")
        print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
        data_format_file = get_from_user_dataframe_format_file(datasets[tabular_data_file])
        if data_format_file:
            # (above condition avoids adding empty views)
            _file_name = os.path.basename(tabular_data_file)
            data_format_files[_file_name] = data_format_file
        
    return data_format_files


In [31]:
data_format_file = get_from_user_multi_view_dataset_fromat_file(single_view_dataset)

NameError: name 'single_view_dataset' is not defined

data_fromat_ref (read only)

CLI editer data_format_file

review : 
- specify lower / upper bound NUMERICAL
- Specify categorical (BOOLEAN, CHARACTER, NUMERICAL)

- save different categorical values 
 a posteriori ex SEX -> male or female, NOT FEMALE
- 

In [57]:
multi_data_format_file = get_from_user_multi_view_dataset_fromat_file(multi_view_dataframe)


++++++++++++++++++++++++++++++++++++++++++++++++++++++
+++++++ Now parsing view: file1 +++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++
displaying first 10 values of feature a (n_feature: 1/18)
0    48
1    87
2    46
3    84
4    94
5    18
6    15
7    30
8    54
9    46
Name: a, dtype: int64
number of differents samples: 57 / total of samples: 100
specify data type for a:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) CUSTOM 
6) UNKNOWN 
7) ignore this column
7
Ignoring feature a
displaying first 10 values of feature e (n_feature: 2/18)
0    98
1    83
2    73
3    45
4    84
5     5
6    44
7    55
8    37
9     8
Name: e, dtype: int64
number of differents samples: 65 / total of samples: 100
specify data type for e:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) CUSTOM 
6) UNKNOWN 
7) ignore this column
7
Ignoring feature e
displaying first 10 values of feature i (n_feature: 3/18)
0    65
1    13
2    81
3    81
4     0
5    57
6    14
7    98
8   

specify data type for discrete:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) CUSTOM 
6) UNKNOWN 
7) ignore this column
7
Ignoring feature discrete
displaying first 10 values of feature city (n_feature: 2/3)
0        Lille
1        Lille
2        Paris
3        Paris
4        Lille
5        Lille
6        Paris
7        Paris
8    Marseille
9        Lille
Name: city, dtype: object
number of differents samples: 3 / total of samples: 100
specify data type for city:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) CUSTOM 
6) UNKNOWN 
7) ignore this column
7
Ignoring feature city
displaying first 10 values of feature pkey (n_feature: 3/3)
0    qpqorfhylu gmfjy bdj
1    kkmjozalfyirgsire ui
2    ezfasuuycdda foisjte
3    faxiqkt xggzmwzoidbg
4    znwhlj rwzdutnagwasy
5    ki vn mtcclzwskgaewy
6    exrmjfolirbtkgjswcin
7    jsgohdb eieenvdwjopr
8    kzrybjgjxm rde xqmra
9    ruchbfa zwgenxslegrl
Name: pkey, dtype: object
number of differents samples: 100 / total of samples

In [58]:
multi_data_format_file

{'file1': {'1': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': True,
   'data_amputation_method': 'MODE_IMPUTATION'},
  'time': {'data_format': 'DATETIME',
   'data_type': 'DATETIME',
   'values': 'object',
   'is_missing_values': False,
   'data_amputation_method': 'MODE_IMPUTATION'},
  'gender': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': True,
   'data_amputation_method': 'KNN_IMPUTATION'},
  'pkey': {'data_format': 'KEY',
   'data_type': 'DATETIME',
   'values': 'object',
   'is_missing_values': False,
   'data_amputation_method': 'KNN_IMPUTATION'}},
 'contatct': {},
 'file2': {}}

In [116]:
save_format_file_ref(multi_data_format_file, 'multi_format_file')

Model successfully saved at multi_format_file


In [48]:
# saving data format file

json_file_name = "format_file_ref"

with open(json_file_name, "w") as format_file:
    json.dump(data_format_file, format_file)

In [7]:
def save_format_file_ref(format_file_ref: Dict[str, Dict[str, Any]], path: str):
    # save `format_file_ref` into a JSON file
    with open(path, "w") as format_file:
        json.dump(format_file_ref, format_file)
    print(f"Model successfully saved at {path}")

In [12]:
def load_format_file_ref(path: str) -> Dict[str, Dict[str, Any]]:
    # retrieve data format file
    with open(path, "r") as format_file:
        format_file_ref = json.load(format_file)
    return format_file_ref

In [6]:

json_file_name = "format_file_ref"


format_file = load_format_file_ref(json_file_name)

In [10]:
format_file

{'pseudo_adni_mod.csv': {'CDRSB.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'int64',
   'is_missing_values': True},
  'ADAS11.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  'MMSE.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'RAVLT.immediate.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': True},
  'RAVLT.learning.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'FAQ.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'TAU.MEDIAN.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'AGE': {'data_format': 'CATEGORICAL',
   'data_t

In [25]:
create_msg_action_selection(DataTypeProperties.CATEGORICAL)

('1)data_type\n2) Values taken\n3) Cancel Operation\n', 3)

In [45]:
ask_for_data_type()

1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) CUSTOM 
6) UNKNOWN 
7) ignore this column
4


{'data_format': 'DATETIME',
 'values': ["<class 'pandas._libs.tslibs.timestamps.Timestamp'>",
  "<class 'pandas._libs.tslibs.timedeltas.Timedelta'>",
  "<class 'pandas._libs.tslibs.period.Period'>",
  "<class 'datetime.datetime'>",
  "<class 'numpy.datetime64'>"]}

In [53]:
import dateutil
from dateutil.parser._parser import ParserError


def create_msg_action_selection(data_type_propreties: Enum) -> Tuple[str, int, Dict[int, Callable]]:
    # create edit selection message for user given data type
    # number of possible action depend of data type properties
    msg = ""
    action_counter = 1
    actions = {}
    
    # data type change command
    msg += "%d) data_type\n" % action_counter
    actions[str(action_counter)] = ask_for_data_type
    action_counter += 1
    
    if data_type_propreties.lower_bound:
        # lower bound edit command
        msg += "%d) lower bound\n" % action_counter
        actions[str(action_counter)] = ask_for_lower_bound
        action_counter += 1
        
    if data_type_propreties.upper_bound:
        # upper bound edit command
        msg += "%d)upper bound\n" % action_counter
        actions[str(action_counter)] = ask_for_upper_bound
        action_counter += 1
        
    if data_type_propreties.set_of_values:
        # value taken edit command
        msg += "%d) Values taken\n" % action_counter
        actions[str(action_counter)] = ask_for_categorical_values
        action_counter += 1
        
    if data_type_propreties.date_format:
        # date formatter edit command
        msg += "%d) Date format\n" % action_counter
        actions[str(action_counter)] = ask_for_date_format
        action_counter += 1
        
    msg += "%d) Cancel Operation\n" % action_counter
    actions[str(action_counter)] = cancel_operation
    return msg, action_counter, actions

def select_action(
                  action: str,
                  possible_actions: Dict[str, Callable],
                  ) -> Tuple[Dict[str, Any], bool]:
    
    # variable initialization
    is_cancelled = False
    new_field = None
    _action_counter = 1  # integer for dynamic cli management
    print('action', action, str(len(possible_actions.keys())),possible_actions)
    if action == str(len(possible_actions.keys())):
        is_cancelled = True
        
    else:
        # define action among the pool of possible actions
        _action_func = possible_actions[action]
        new_field = _action_func()
    
    return new_field, is_cancelled


def isfloat(value:str) ->bool:
    """checks if string represents a float or int"""
    is_float = True
    try:
        float(value)
    except ValueError as e:
        is_float = False
    return is_float



def is_datetime(date: str) -> bool:
    """checks if date is a date"""
    is_date_parsable = True
    try:
        dateutil.parser.parse(date)
    except (ParserError, ValueError) as err:
        is_date_parsable = False
        
    return is_date_parsable

def cancel_operation():
    print("operation cancelled")
        
def ask_for_lower_bound() -> Dict[str, float]:
    _is_entered_value_correct = False
    while not _is_entered_value_correct:
        lower_bound = input('enter lower bound')
        if isfloat(lower_bound) or is_datetime(lower_bound):
            # check if entered value is correct (is a numerical value)
            _is_entered_value_correct = True
        else:
            print('Value not a Number! please retry')
    return {'lower_bound': float(lower_bound)}

def ask_for_upper_bound() -> Dict[str, float]:
    
    _is_entered_value_correct = False
    while not _is_entered_value_correct:
        upper_bound = input('enter upper bound')
        if isfloat(upper_bound) or is_datetime(upper_bound):
            # check if entered value is correct (is a numerical value)
            _is_entered_value_correct = True
        else:
            print('Value not a Number! please retry')
    return {'upper_bound': float(upper_bound)}


def _ask_for_data_type(data_type: Enum) -> Enum:
    """asks user for datatype contains in `data_type`
    If user selects `cancel`, it will return None
    """
    
    _available_data_type = [t for t in data_type]  # get all keys contain in data_type
    n_avail_data_type = len(_available_data_type)
    msg, _n_answer = get_data_type_selection_msg(data_type, ign_msg="cancel operation")
    data_type_selection = get_user_input(msg, _n_answer)
    
    if str(_n_answer) != data_type_selection:
        return _available_data_type[int(data_type_selection) - 1]
    
    
def ask_for_data_type() -> Dict[str, Any]:

    updates = None
    
    new_data_format = _ask_for_data_type(DataType)
    
    if new_data_format is not None: 
        # case where 'cancel operation' hasnot been selected
                
        updates = {'data_format': new_data_format.name}
        if  isinstance(next(iter(new_data_format.value)), Enum):
            # if subtypes are available
            new_data_type= _ask_for_data_type(new_data_format.value)
            new_values = list(map(lambda x: str(x), new_data_type.value))
            updates.update({'data_type': new_data_type, 'values': new_values})
        else:
            new_values = list(map(lambda x: str(x), new_data_format.value))
            updates.update({'values': new_values})
    return updates


def ask_for_categorical_values() -> Dict[str, Any]:
    possible_values = input('enter possible values (separated by ",")')
    possible_values = possible_values.split(",")  # separate values passed by user into a list
    return {'categorical_values': possible_values}


def ask_for_date_format() -> Dict[str, Any]:
    # TODO : ask for date format (UTC, ....)
    msg = 'please enter date format:\n1)timetsamp\n2)ISO date format (YYYY-MM-DD)\n3)custom date format\n'
    user_selection = input(msg)
    # default date format
    msg = {'1': 'timestamp',
          '2': '(American default date format) mm/dd/yy',
          '3': '(Europeen default date format) dd/mm/yy',
           '4': 'ISO date format (YYYY-MM-DD)',
          '5': 'custom date format',
          '6': 'select timezone'}
    pass

def edit_feature_format_file_ref(feature_content: Dict[str, Any],
                                  feature_name: str,
                                  available_categorical_data_type: List[Enum],
                                  messages: Dict[str, str],
                                  ignore_keystroke: int) -> Dict[str, Any]:
    """Edits a specific feature that belongs to a specific view within a format file"""
    

    _is_feature_unparsed = True  
    _is_cancelled = False  # whether parsing of current column has been cancelled or not
    _is_first_edit = True
    _avail_data_type_properties = [dtype for dtype in DataTypeProperties]
    
    # iterate over number of feature contained in view, and ask for each feature if changes are needed
    while _is_feature_unparsed:
        if _is_cancelled or not _is_first_edit:
            _f_answer = True
        else:
            _f_answer = get_user_input(f"Edit variable: {feature_name}?\n" + messages['yes_or_no'], 2)
            # ask if user wants to edit feature variables
            _f_answer = parse_yes_no_msg(_f_answer)
            _is_operation_cancelled = False  # for cancelling feature edition
            _is_first_edit = False
        if _f_answer:
            # case where user wants to edit the current feature
            
            _msg = messages['edit']
            
            data_format = feature_content.get('data_format')
            for data_type_properties in _avail_data_type_properties:
                if data_format == data_type_properties.name:
                    # get data property from data_format
                    select_msg, n_actions, possible_actions = create_msg_action_selection(data_type_properties)
                    _msg += select_msg
                    _edit_selection = get_user_input(_msg, n_actions)

                    _edited_field, _is_cancelled = select_action(
                                                                  _edit_selection,
                                                                  possible_actions
                                                                  #available_categorical_data_type,
                                                                  #messages['data_type_select']
                                                                )

            if not _is_cancelled:
                # if user has not cancelled field edition
                if _edited_field is not None:
                    feature_content.update(_edited_field)
             
                _c_answer = get_user_input(f"Continue Editing variable: {feature_name}?\n" + messages['yes_or_no'], 2)
                _is_feature_unparsed = parse_yes_no_msg(_c_answer)
            else:
                _is_feature_unparsed = False
                
        else:
            _is_feature_unparsed = False
            
    return feature_content

In [83]:
 CategoricalDataType.BOOLEAN.name
def edit_format_file_ref(format_file_ref: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    
    # CLI for editing `format_file_ref`, a file containing information about each variable
    # in a tabular dataset
    print(f'Now editing format file ref')
    
    ## variables initialization
    available_categorical_data_types = [t for t in CategoricalDataType]
    _file_names = list(format_file_ref.keys())
    _n_tot_files = len(_file_names)
    
    ## messages definition
    _data_type_selection_msg, ign_key = get_data_type_selection_msg(available_categorical_data_types)
    
    _messages = {
        'yes_or_no': get_yes_no_msg(),
        'edit': 'Which field should be modified?\n',
    }

    
    
    # iterate over name of files (ie views)
    for i_file in range(_n_tot_files):
        # ask for each file if user wants to edt it
        _answer = get_user_input(f"Edit file: {_file_names[i_file]}?\n" + _messages['yes_or_no'], 2)
        _answer = parse_yes_no_msg(_answer)
        
        if _answer:
            # case where user wants to modify current view scheme
            _file_content = format_file_ref[_file_names[i_file]]  # get file (ie view) content
            
            ## variables initialization for parsing current view
            _features_names = list(_file_content.keys())
            _n_tot_feature = len(_features_names)
            
            # iterate over features found in view
            for i_feature in range(_n_tot_feature):
                feature_name = _features_names[i_feature]
                feature_content = _file_content[feature_name]
                feature_content = edit_feature_format_file_ref(feature_content,
                                                               feature_name,
                                                               available_categorical_data_types,
                                                               _messages,
                                                               ign_key)
            format_file_ref[_file_names[i_file]].update({feature_name: feature_content})
            
    return format_file_ref

In [90]:
format_file

{'pseudo_adni_mod.csv': {'CDRSB.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': ["<class 'bool'>"],
   'is_missing_values': True,
   'categorical_values': ['1', ' 2', ' 4']},
  'ADAS11.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  'MMSE.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'RAVLT.immediate.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': True},
  'RAVLT.learning.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'FAQ.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False,
   'lower_bound': 0.0},
  'TAU.MEDIAN.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is

In [ ]:
edit_format_file_ref(format_file)

# TODO: create possiblity for user to update data type when editing format_file

In [85]:
multi_format_file_ref = load_format_file_ref('multi_format_file')
edit_format_file_ref(multi_format_file_ref)

Now editing format file ref
Edit file: file1?
1) YES
2) NO
2
Edit file: contatct?
1) YES
2) NO
1
Edit variable: discrete?
1) YES
2) NO
1
Which field should be modified?
1) data_type
2) lower bound
3)upper bound
4) Cancel Operation
1
action 1 4 {'1': <function ask_for_data_type at 0x7ff11b38e3a0>, '2': <function ask_for_lower_bound at 0x7ff11b456ca0>, '3': <function ask_for_upper_bound at 0x7ff11b4569d0>, '4': <function cancel_operation at 0x7ff11b456160>}
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) CUSTOM 
6) UNKNOWN 
7) cancel operation
5
1) DISCRETE 
2) CHARACTER 
3) cancel operation
2
Continue Editing variable: discrete?
1) YES
2) NO
2
Edit variable: city?
1) YES
2) NO
1
Which field should be modified?
1) data_type
2) Values taken
3) Cancel Operation
2
action 2 3 {'1': <function ask_for_data_type at 0x7ff11b38e3a0>, '2': <function ask_for_categorical_values at 0x7ff11b38e8b0>, '3': <function cancel_operation at 0x7ff11b456160>}
enter possible values (separated by ",")a,

{'file1': {'e': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  '1': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': False},
  '2': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': True},
  'time': {'data_format': 'DATETIME',
   'data_type': 'DATETIME',
   'values': 'object',
   'is_missing_values': False},
  'pressure': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'e.1': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'gender': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': False},
  'blood type': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': True},
  '

## tabular data sanity check using file format ref

In [9]:
    



# utility functions for multi view dataframe
def rename_variables_before_joining(multi_view_datasets: Dict[str, pd.DataFrame],
                                    views_name: List[Union[str, int]],
                                    primary_key:Union[str, int]=None) -> Tuple[Dict[str, pd.DataFrame]]:
    """
    Renames variables that have same name but different views using the following naming convention:
    if `a` is the name of a feature of `view1` and `a` is the name of a feature of `view2`,
    features names will be updated into `view1.a` and `view2.a`
    """
    _features_names = {}
    _views_length = len(views_name)
    
    # check for each variable name existing in one view, that it doesnot exist in another
    # view. if it is, rename both variables
    # for this purpose, parse every combination once
    for i_left in range(0, _views_length-1):
        _left_view = views_name[i_left]
        _left_features_name = multi_view_datasets[_left_view].columns.tolist()
        for i_right in range(i_left+1, _views_length):
        
            _right_view = views_name[i_right]
            _right_features_name = multi_view_datasets[_right_view].columns.tolist()
            
            for _f in _left_features_name:
                if primary_key and _f == primary_key:
                    # do not affect primary key (if any)
                    continue
                if _f  in _right_features_name:
                    
                    if _left_view  not in _features_names:
                        _features_names[_left_view] = {}
                        
                    if _right_view not in _features_names:
                        _features_names[_right_view] = {}
                        
                    _features_names[_left_view].update({_f: _left_view + '.' + str(_f)})
                    _features_names[_right_view].update({_f: _right_view + '.' + str(_f)})
    
    for i in range(_views_length):
        _view = views_name[i]
        _new_features = _features_names.get(_view)
        if _new_features:
            multi_view_datasets[_view] = multi_view_datasets[_view].rename(columns=_new_features)
        
    
    return multi_view_datasets


def create_multi_view_dataframe(datasets: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    _header_labels = ['views', 'feature_name']
    # 1. create multiindex header

    _feature_name_array = np.array([])  # store all feature names
    _view_name_array = []  # store all views (ie modalities) names

    _concatenated_datasets = np.array([])  # store dataframe values

    for key in datasets.keys():
        _feature_name_array = np.concatenate([_feature_name_array,
                                              datasets[key].columns.values])
        if len(_concatenated_datasets) <= 0:
            # first pass 
            _concatenated_datasets = datasets[key].values
        else:
            # next passes
            try:
                _concatenated_datasets = np.concatenate(
                                        [_concatenated_datasets,
                                         datasets[key].to_numpy()
                                         ], axis=1)
            except ValueError as val_err:
                # catching case where nb_samples are differents
                raise ValueError(
                    'Cannot create multi view dataset: different number of samples for each modality have been detected'\
                        + 'Details: ' + str(val_err)
                    )
        for _ in datasets[key].columns.values:
            _view_name_array.append(key)

    _header = pd.MultiIndex.from_arrays([_view_name_array,
                                         _feature_name_array],
                                        names=_header_labels)


    # 2. create multi index dataframe

    multi_view_df = pd.DataFrame(_concatenated_datasets,
                                  columns = _header)
    return multi_view_df


def join_muti_view_dataset(multi_view_dataset: pd.DataFrame,
                           primary_key: str=None) -> pd.DataFrame:
    """Concatenates a multi view dataset into a plain pandas dataframe,
    by doing a join operation along specified primary_key"""
    _views_name = sorted(set(multi_view_dataset.columns.get_level_values(0)))  # get views name
    
    joined_dataframe = multi_view_dataset[_views_name[0]]  # retrieve the first view
    # (as a result of join operation)
    for x in range(1, len(_views_name)):
        joined_dataframe = joined_dataframe.merge(multi_view_dataset[_views_name[x]],
                                                    on=primary_key,
                                                    suffixes=('', '.'+_views_name[x]))
        
        #df['file1'].join(df['file2'].set_index('pkey'), on='pkey', rsuffix='.file2')
        
    return joined_dataframe


def search_primary_key(format_file_ref: Dict[str, Dict[str, Any]]) -> Optional[str]: 
    """"""
    _views_names = list(format_file_ref.keys())
    primary_key = None
    _c_view = None
    for view_name in _views_names:
        file_content = format_file_ref[view_name]
        _features_names = list(file_content.keys())
        for feature_name in _features_names:
            feature_content  = file_content[feature_name]
            _d_format = feature_content.get('data_format')
            
            if _d_format == DataType.KEY.name:
                if _c_view is None:
                    primary_key = feature_name
                    _c_view = view_name
                    print(f'found primary key {primary_key}')
                else:
                    print(f'error: found 2 primary keys is same view {view_name}')
        _c_view = None
    return primary_key



def select_data_from_format_file_ref(datasets: Dict[str, Dict[str, Any]],
                                     format_file: Dict[str, Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """returns an updated dataset containing only the features detailed in format_file"""
    # variables initialisation
    
    updated_dataset = {}
    _views_format_file = list(format_file.keys())
    
    for view in _views_format_file:
        if view in datasets.keys():
            # only extract features from format_file
            _format_file_features = list(format_file[view].keys())
            _current_dataset_feature = datasets[view].columns.tolist()
            try:
                updated_dataset[view] = datasets[view][_format_file_features]
            except KeyError as ke:
                # catch error if a column is specified in data format file
                # but not found in dataset
                _missing_feature = []
                for feature in _format_file_features:
                    if feature not in _current_dataset_feature:
                        _missing_feature.append(feature)
                print('Error: th following features', *_missing_feature, f'are not found in view: {view}')
        else:
            # trigger error
            print(f'error!: missing view {view} in dataset')
            
    return updated_dataset

In [20]:
df_to_check[n_feature_name]dataset_to_check = load_tabular_datasets(r'/user/ybouilla/home/Documents/data/pseudo_adni_mod/pseudo_adni_mod.csv')



file found


In [22]:
# extract views names
views_names = list(format_file.keys())



# look for primary key
primary_key = search_primary_key(format_file)
print('primary key', primary_key)

# select only features in dataset that will be checked
pre_parsed_dataset_to_check = select_data_from_format_file_ref(dataset_to_check, format_file)
# rename columns names before join operation
pre_parsed_dataset_to_check = rename_variables_before_joining(pre_parsed_dataset_to_check, views_names)
pre_parsed_dataset_to_check

multi_df_to_check = create_multi_view_dataframe(pre_parsed_dataset_to_check)
multi_df_to_check

#if primary_key is not None:
# jointure operation (takesplace only if primary key has been specfied in foramt_file)
df_to_check = join_muti_view_dataset(multi_df_to_check)
    
df_to_check

primary key None


feature_name,CDRSB.bl,ADAS11.bl,MMSE.bl,RAVLT.immediate.bl,RAVLT.learning.bl,FAQ.bl,TAU.MEDIAN.bl,AGE
0,1.0,8.0,27.0,23.739439,4.0,3.0,132.571916,75.0
1,0.0,0.0,30.0,64.933800,9.0,0.0,33.787719,67.0
2,0.0,8.0,24.0,36.987722,3.0,0.0,110.049924,63.0
3,0.0,3.0,29.0,50.314425,5.0,3.0,138.690457,75.0
4,0.0,0.0,30.0,57.217830,9.0,0.0,-61.573234,65.0
...,...,...,...,...,...,...,...,...
995,1.0,2.0,29.0,61.896022,8.0,0.0,87.065806,76.0
996,0.0,1.0,29.0,62.083170,8.0,1.0,121.985248,77.0
997,3.0,14.0,24.0,22.289059,2.0,7.0,157.229102,74.0
998,0.0,13.0,26.0,31.650504,2.0,4.0,103.238647,64.0


In [80]:
# sanity check on columns using format_file



_views_format_file = list(format_file.keys())
    
for view in _views_format_file:
    _features_format_file = format_file[view]
    
    for feature in _features_format_file:
        feature_content = df_to_check[feature]
        feature_format =  _features_format_file[feature]
        # 1. check for datatype consistency
        
        data_types = find_data_type(feature_format['data_format'], feature_format['data_type'])
        
        print(dtype[-1].value, feature_content.dtype, feature_format)
        if any(t == feature_content.dtype for t in data_types[-1].value ):
            print('ok')
        


[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'CATEGORICAL', 'data_type': 'BOOLEAN', 'values': ["<class 'bool'>"], 'is_missing_values': True, 'categorical_values': ['1', ' 2', ' 4']}
[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'QUANTITATIVE', 'data_type': 'DISCRETE', 'values': 'int64', 'is_missing_values': True}
[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'CATEGORICAL', 'data_type': 'NUMERICAL', 'values': 'float64', 'is_missing_values': False}
ok
[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'QUANTITATIVE', 'data_type': 'CONTINUOUS', 'values': 'float64', 'is_missing_values': True}
ok
[<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>] float64 {'data_format': 'CATEGORICAL', 'data_type': 'NUMERICAL', 'values': 'float64', 'i

In [10]:
multi_format_file_ref = load_format_file_ref('multi_format_file')
multi_dataset_to_check = load_tabular_datasets(r'test7')

NameError: name 'load_format_file_ref' is not defined

In [14]:
multi_format_file_ref['file1']['2']

{'data_format': 'CATEGORICAL',
 'data_type': 'BOOLEAN',
 'values': 'bool',
 'is_missing_values': True}

In [91]:
# extract views names
views_names = list(multi_format_file_ref.keys())



# look for primary key
primary_key = search_primary_key(multi_format_file_ref)
print('primary key', primary_key)

# select only features in dataset that will be checked
pre_parsed_dataset_to_check = select_data_from_format_file_ref(multi_dataset_to_check, multi_format_file_ref)
# rename columns names before join operation
pre_parsed_dataset_to_check = rename_variables_before_joining(pre_parsed_dataset_to_check, views_names,
                                                             primary_key)
pre_parsed_dataset_to_check

multi_df_to_check = create_multi_view_dataframe(pre_parsed_dataset_to_check)
multi_df_to_check

#if primary_key is not None:
# jointure operation (takesplace only if primary key has been specfied in foramt_file)
df_to_check = join_muti_view_dataset(multi_df_to_check, primary_key)
    
df_to_check


found primary key pkey
found primary key pkey
found primary key pkey
primary key pkey


feature_name,discrete,city,pkey,e,file1.1,2,file1.time,pressure,e.1,gender,blood type,file2.1,file2.time,pH
0,64.0,Lille,qpqorfhylu gmfjy bdj,16,False,False,2018-01-03 04:00:00,0.98667,98,WOMAN,A,False,2018-01-02 06:00:00,NaN
1,26.0,Lille,kkmjozalfyirgsire ui,96,True,True,2018-01-02 04:00:00,0.996889,35,MAN,AB,True,2018-01-01 00:00:00,0.023107
2,61.0,Paris,ezfasuuycdda foisjte,8,True,True,2018-01-01 09:00:00,0.777026,65,MAN,A,False,2018-01-02 10:00:00,0.587685
3,29.0,Paris,faxiqkt xggzmwzoidbg,6,True,False,2018-01-04 20:00:00,0.877527,81,MAN,AB,True,2018-01-03 12:00:00,0.894073
4,99.0,Lille,znwhlj rwzdutnagwasy,79,True,True,2018-01-04 09:00:00,0.447389,88,WOMAN,O,True,2018-01-01 10:00:00,0.026831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,9.0,Paris,zeqhcikzdodus jn qjf,62,True,True,2018-01-02 13:00:00,0.953184,53,MAN,AB,False,2018-01-02 05:00:00,0.78856
96,98.0,Marseille,iicthcvfmkajbvr gzir,49,False,True,2018-01-02 21:00:00,0.442283,35,MAN,NaN,True,2018-01-05 02:00:00,0.402979
97,21.0,Lille,ztjakcsk bhjoksdz lm,14,False,False,2018-01-02 06:00:00,0.988543,67,MAN,B,False,2018-01-01 12:00:00,NaN
98,42.0,Marseille,sabunaa opt vpulnxj,10,True,False,2018-01-02 01:00:00,0.059791,48,MAN,B,True,2018-01-02 09:00:00,0.651801


{'file1': ['e',
  'file1.1',
  '2',
  'file1.time',
  'pressure',
  'e.1',
  'gender',
  'blood type',
  'pkey'],
 'contatct': ['discrete', 'city', 'pkey'],
 'file2': ['file2.1', 'file2.time', 'pH', 'pkey']}

In [105]:
def check_key_variable_compliance(column: pd.Series,
                                  
                                  col_name:str=None,
                                  warning=None) -> bool:
    """performs data sanity check over variable of type `KEY`
    warning should be Critical warnings
    """
    # variables initialisation
    is_test_passed = True 
    
    # 1. check unicity of values in column
    
    n_unique_samples = unique(column, number=True)
    n_samples = column.shape[0]
    
    if n_unique_samples != n_samples:
        is_test_passed = False
        print(f'error: keys not unique ! b of samples= {n_samples} and unique values {n_unique_samples}')
    else:
        print('test 1 passed')
    # 2. check if missing database contained in key (key should not contain any missing data)
    if check_missing_data(column):
        is_test_passed = False
        print('error: missing data found in key')
    else:
        
        print('test 2 passed')

                
    return is_test_passed




In [47]:
np.all(df_to_check['file1.time'] > '2018-01-04 09:00:00')

False

In [57]:
np.all(df_to_check['file1.time'].apply(is_datetime))

True

In [50]:
check_variable_compliance(df_to_check['2'], multi_format_file_ref['file1']['2'])



test 1 passed
test 2 passed
test 3 skipped 
test 4 skipped
test 5: passed


In [35]:
multi_format_file_ref['file1']['2'].update({'categorical_values': [True,False]})

In [36]:
multi_format_file_ref['file1']['2']

{'data_format': 'CATEGORICAL',
 'data_type': 'BOOLEAN',
 'values': 'bool',
 'is_missing_values': True,
 'categorical_values': [True, False]}

In [100]:
def check_variable_compliance(column: pd.Series,
                               format_file_ref: Dict[str, Any],
                               col_name:str=None,
                               warning=None) -> Tuple[bool, bool]:
    """performs a data sanity check on variable `col_name` given instruction in 
    data_file_ref
    """
    is_test_passed = True
    
    
    data_format_name = format_file_ref.get('data_format')
    data_type_name = format_file_ref.get('data_type')
    # remove nan (missing values) from 
    column_without_nan = column.dropna()
    
    
    if data_format_name is None:
        print(f'critical wraning: data fromat {data_format_name} not understood')
    # 1. check data sub type
    try:
        data_type = find_data_type(data_format_name, data_type_name)
    except ValueError as err:
        data_type = None
        print('Critical warning: data format and data type mismatch')
    does_column_have_correct_data_type = any(t for t in data_type.value)
    if not does_column_have_correct_data_type:
        print(f'error: data type {column.dtype} doesnot have the data type specified in format reference file')
    else:
        print('test 1 passed')

    # 2. check if missing values are allowed
    is_missing_data = check_missing_data(column)
    is_missing_values_authorized = format_file_ref.get('is_missing_values', 'test_skipped')
    if is_missing_values_authorized == 'test_skipped':
        print('missing_value test skipped')
    elif not is_missing_values_authorized and is_missing_data:

        print('Error found missing data but missing data are not authorized')
    else:
        print('test 2 passed')
    
    
    # 3. check lower bound
    print(format_file_ref)
    lower_bound = format_file_ref.get('lower_bound')
    
    if lower_bound is not None:
        
        # should work for both numerical and datetime data sets
        
        is_lower_bound_correct = np.all(column_without_nan >= lower_bound)
        
            
        if not is_lower_bound_correct:
            print('Warning: found some data below lower bound')
        else:
            print('test 3 passed')
    else:
        print('test 3 skipped ')
    # 4. check upper bound
    upper_bound = format_file_ref.get('upper_bound')
    if upper_bound is not None:
         # should work for both numerical and datetime data sets
        is_upper_bound_correct = np.all(column_without_nan <= lower_bound)
        
            
        if not is_upper_bound_correct:
            print('Warning: found some data  above upper bound')
        else:
            print('test 4 passed')
            
    else:
        print('test 4 skipped')
    # 5. check if possible_values are contained in variable
    categorical_values = format_file_ref.get('categorical_values')    
    if categorical_values is None:
        print('categorical value check test skipped')
    else:
        unique_values = unique(column)
        _is_error_found = False
        for val in unique_values:
            if val not in categorical_values and not np.isnan(val):
                print(f'critical warning: {val} not in possible values')
                _is_error_found = True
        if not _is_error_found:
            print('test 5: passed')
    
 
        
    
def check_datetime_variable_compliance(column: pd.Series):
    """additional data sanity checks for datetime variable"""
    # test 1. check if datetime is parsable
    
    # remove nan
    column_without_nan = column.dropna()
    are_datetime_parsables =  np.all(column.apply(is_datetime))
    if not are_datetime_parsables:
        print('Warning: at least one variable is not a datetime')
        
    else:
        print('datetime parsed')

In [106]:
# Data sanity check

new_feature_name = { v: list(pre_parsed_dataset_to_check[v].columns) for v in views_names}
new_feature_name

for view in views_names:
    print(view)
    
    feature_names = list(multi_format_file_ref[view].keys())
    for n_feature_name, feature_name in zip(new_feature_name[view], feature_names):
        check_variable_compliance(df_to_check[n_feature_name], multi_format_file_ref[view][feature_name])
        data_format = multi_format_file_ref[view][feature_name].get('data_format')
        if data_format == DataType.DATETIME.name:
            # addtional check for DATETIME data format
            check_datetime_variable_compliance(df_to_check[n_feature_name])
            
        if data_format == DataType.KEY.name:
            check_key_variable_compliance(df_to_check[n_feature_name])
            
        

file1
test 1 passed
test 2 passed
{'data_format': 'QUANTITATIVE', 'data_type': 'DISCRETE', 'values': 'int64', 'is_missing_values': True}
test 3 skipped 
test 4 skipped
categorical value check test skipped
test 1 passed
test 2 passed
{'data_format': 'CATEGORICAL', 'data_type': 'BOOLEAN', 'values': 'bool', 'is_missing_values': False}
test 3 skipped 
test 4 skipped
categorical value check test skipped
test 1 passed
test 2 passed
{'data_format': 'CATEGORICAL', 'data_type': 'BOOLEAN', 'values': 'bool', 'is_missing_values': True, 'categorical_values': [True, False]}
test 3 skipped 
test 4 skipped
test 5: passed
test 1 passed
test 2 passed
{'data_format': 'DATETIME', 'data_type': 'DATETIME', 'values': 'object', 'is_missing_values': False}
test 3 skipped 
test 4 skipped
categorical value check test skipped
datetime parsed
test 1 passed
test 2 passed
{'data_format': 'QUANTITATIVE', 'data_type': 'CONTINUOUS', 'values': 'float64', 'is_missing_values': False}
test 3 skipped 
test 4 skipped
categor

In [71]:
multi_format_file_ref['file2']['pH'].update({'lower_bound': -10})
multi_format_file_ref['file2']['pH']

{'data_format': 'QUANTITATIVE',
 'data_type': 'CONTINUOUS',
 'values': 'float64',
 'is_missing_values': False,
 'lower_bound': -10}

In [97]:
multi_format_file_ref[view][feature_name]

{'data_format': 'DATETIME',
 'data_type': 'DATETIME',
 'values': 'object',
 'is_missing_values': False}

In [ ]:
find_data_type(

In [80]:
check_variable_compliance(df_to_check['pH'], multi_format_file_ref['file2']['pH'])

test 1 passed
Error found missing data but missing data are not authorized
{'data_format': 'QUANTITATIVE', 'data_type': 'CONTINUOUS', 'values': 'float64', 'is_missing_values': False, 'lower_bound': -10}
test 3 passed
test 4 skipped
categorical value check test skipped


In [77]:
df_to_check['pH'] > np.nan

0     False
1     False
2     False
3     False
4     False
      ...  
95    False
96    False
97    False
98    False
99    False
Name: pH, Length: 100, dtype: bool

In [66]:
multi_format_file_ref

{'file1': {'e': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  '1': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': False},
  '2': {'data_format': 'CATEGORICAL',
   'data_type': 'BOOLEAN',
   'values': 'bool',
   'is_missing_values': True,
   'categorical_values': [True, False]},
  'time': {'data_format': 'DATETIME',
   'data_type': 'DATETIME',
   'values': 'object',
   'is_missing_values': False},
  'pressure': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'e.1': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'gender': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'object',
   'is_missing_values': False},
  'blood type': {'data_format': 'CATEGORICAL',
   'data_type': 'CHARACTER',
   'values': 'obj

In [19]:
check_variable_compliance(df_to_check['pkey'], multi_format_file_ref['file2']['pkey'])

test 1 passed
test 2 passed
categorical value check test skipped


## data_format_file

In [33]:
format_file

{'pseudo_adni_mod.csv': {'CDRSB.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'int64',
   'is_missing_values': True},
  'ADAS11.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': True},
  'MMSE.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'RAVLT.immediate.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': True},
  'RAVLT.learning.bl': {'data_format': 'CATEGORICAL',
   'data_type': 'NUMERICAL',
   'values': 'float64',
   'is_missing_values': False},
  'FAQ.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'DISCRETE',
   'values': 'int64',
   'is_missing_values': False},
  'TAU.MEDIAN.bl': {'data_format': 'QUANTITATIVE',
   'data_type': 'CONTINUOUS',
   'values': 'float64',
   'is_missing_values': False},
  'AGE': {'data_format': 'CATEGORICAL',
   'data_t

In [33]:
msg_yes_or_no_question

'1) YES\n2) NO\n'

In [40]:
is_views_finished = False


views_format_file = {}

while not is_views_finished:
    is_features_finished = False
    resp = input('do you want to add a new view (file)?\n' + msg_yes_or_no_question)
    resp = yes_or_no_question_key.get(resp)
    if not resp:
        is_views_finished = True
        print('process done')
        continue
    new_view = input('please add new view name:\n')
    while not is_features_finished:
        feature_format_file = {}
        new_feature = input('please add new feature name:\n')
        feature_format_file[new_feature] = {}categorical_values
        is_column_parsed = False
        try:
            while not is_column_parsed:
                data_format_id = input(f'specify data type for {feature}:\n' + msg )
                if data_format_id.isdigit() and int(data_format_id) <= n_available_data_type+1:
                    # check if value passed by user is correct (if it is integer,
                    # and whithin range [1, n_available_data_type])
                    is_column_parsed = True
                
                else:
                    print(f'error ! {data_format_id} value not understood')
                    
        except KeyboardInterrupt as e:
            print('stopping now' + str(e))
        resp = input('do you want to add a new variable (feature) ?' + msg_yes_or_no_question)
        resp = yes_or_no_question_key.get(resp)
        if not resp:
            is_features_finished = True
            print('process done')
            continue
    views_format_file[new_view] = feature_format_file

do you want to add a new view (file)?
1) YES
2) NO
1
please add new view name:
ll
please add new feature name:
ll
specify data type for 0:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) UNKNOWN 
6) ignore this column
4
do you want to add a new variable (feature) ?1) YES
2) NO
2
process done
do you want to add a new view (file)?
1) YES
2) NO
1
please add new view name:
kk
please add new feature name:
vkeof
specify data type for 0:
1) KEY 
2) QUANTITATIVE 
3) CATEGORICAL 
4) DATETIME 
5) UNKNOWN 
6) ignore this column
3
do you want to add a new variable (feature) ?1) YES
2) NO
2
process done
do you want to add a new view (file)?
1) YES
2) NO
2
process done


In [75]:
type(np.datetime64("2018-01-01"))
import datetime
type(datetime.datetime(2018, 1, 1))

pd.datetime64[ns]

AttributeError: module 'pandas' has no attribute 'datetime64'

In [91]:
t = type(pd.to_datetime('13000101', format='%Y%m%d', errors='ignore'))

In [107]:
t = pd.Series(pd.date_range("1/1/2011", freq="H", periods=3)).dtype

t =type(t).type

In [108]:
any(t == t1 for t1 in [pd.Timestamp, pd.Timedelta, pd.Period, datetime.datetime,np.datetime64] )

True

In [105]:
t.type

numpy.datetime64

In [28]:
data_format_file

{'CDRSB.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': int,
  'is_missing_values': False},
 'ADAS11.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': int,
  'is_missing_values': False},
 'MMSE.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': int,
  'is_missing_values': False},
 'RAVLT.immediate.bl': {'data_type': <QuantitativeDataType.CONTINUOUS: [<class 'float'>, <class 'numpy.float64'>]>,
  'values': float,
  'is_missing_values': False},
 'RAVLT.learning.bl': {'data_type': <CategoricalDataType.NUMERICAL: [<class 'float'>, <class 'int'>, <class 'numpy.float64'>, <class 'numpy.int64'>]>,
  'values': float,
  'is_missing_values': False},
 'RAVLT.forgetting.bl': {'data_type': <QuantitativeDataType.CON

In [29]:

type(dataset[feature].dtype)

numpy.dtype[float64]

In [28]:
dir(dataset[feature])

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__r